In [2]:
import numpy as np
import pandas as pd
import torch
import sklearn

In [3]:
torch.manual_seed(1)

In [4]:
train_df = pd.read_csv("drive/MyDrive/Colab Notebooks/data_sample/2021-ai-w4-p1/train.csv")
test_df = pd.read_csv("drive/MyDrive/Colab Notebooks/data_sample/2021-ai-w4-p1/test.csv")
submit_df = pd.read_csv("drive/MyDrive/Colab Notebooks/data_sample/2021-ai-w4-p1/submit.csv")
print(train_df.head())
print(test_df.head())
print(submit_df.head())

   Pregnancies  Glucose  BloodPressure  ...  DiabetesPedigreeFunction  Age  Diabetes
0            1       95             60  ...                     0.260   22         0
1            5      105             72  ...                     0.159   28         0
2            0      135             68  ...                     0.365   24         1
3            4      131             68  ...                     0.160   28         0
4            1      103             30  ...                     0.183   33         0

[5 rows x 9 columns]
   Pregnancies  Glucose  BloodPressure  ...   BMI  DiabetesPedigreeFunction  Age
0            6       98             58  ...  34.0                     0.430   43
1            2      112             75  ...  35.7                     0.148   21
2            2      108             64  ...  30.8                     0.158   21
3            8      107             80  ...  24.6                     0.856   34
4            7      136             90  ...  29.9              

In [5]:
# preprocessing 
train_x = np.array(train_df.drop(['Diabetes'],axis=1))
train_y = np.array(train_df['Diabetes'])
test_x =  np.array(test_df)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_x=scaler.fit_transform(train_x)
test_x= scaler.transform(test_x)


train_x = torch.Tensor(train_x).cuda()
# 0,1 sigmoid 는 float Tensor 이용
train_y = torch.Tensor(train_y).unsqueeze(1).cuda()
test_x = torch.Tensor(test_x).cuda()

In [7]:
w = torch.zeros((8,1),requires_grad=True,device="cuda")
b = torch.zeros(1,requires_grad=True,device="cuda")

optim = torch.optim.SGD([w,b],lr = 1e-3)

epochs = 10000

loss = torch.nn.BCELoss()


In [8]:
for epoch in range(epochs+1):
  optim.zero_grad()

  h = torch.sigmoid(train_x.matmul(w) +b)
  cost = loss(h,train_y)

  cost.backward()
  optim.step()
  if epoch % 1000 == 0:
    print(cost.item())

0.6931471824645996
0.5982561111450195
0.5515334606170654
0.5249934196472168
0.5082526206970215
0.49690964818000793
0.48883306980133057
0.4828740060329437
0.478359580039978
0.47486943006515503
0.4721274673938751


In [9]:
with torch.no_grad():
  predict = torch.sigmoid(test_x.matmul(w) + b)
predict = (predict >= torch.Tensor([0.5]).cuda()).int()

submit_df['Diabetes'] = predict.cpu().detach().numpy()
submit_df.head()

,ID,Diabetes
0,0,0
1,1,0
2,2,0
3,3,0
4,4,1


In [11]:
with torch.no_grad():
  h = torch.sigmoid(train_x.matmul(w)+b)
h = (h >= torch.Tensor([0.5]).cuda()).int()
h = h == train_y
print(h.sum().item()/len(train_y))

0.7839851024208566
